In [1]:
import subprocess

import matplotlib.pyplot as plt

import pygimli as pg
from pygimli.meshtools import readGmsh


In [12]:
mesh_file='/bettik/alberta/small-arctic-10km-sidfex/experiments/run2_330d_partnodisk/par32small_arctic_10km.msh'


In [4]:
mesh = readGmsh(mesh_file, verbose=True)

Reading /bettik/alberta/small-arctic-10km-sidfex/experiments/run3_330d_partnodisk/par32small_arctic_10km.msh... 



UnicodeDecodeError: 'utf-8' codec can't decode byte 0x90 in position 57: invalid start byte

In [13]:
import os

import numpy as np

import pygimli as pg

In [14]:
fName=mesh_file
verbose=True
precision=None

In [15]:
    assert precision is None or precision >= 0
    inNodes, inElements, nCount = 0, 0, 0
    fid = open(fName, encoding='latin-1')

In [16]:
    for line in fid:

        if line[0] == '$':
            if line.find('Nodes') > 0:
                inNodes = 1
            if line.find('EndNodes') > 0:
                inNodes = 0
            if line.find('Elements') > 0:
                inElements = 1
            if line.find('EndElements') > 0:
                inElements = 0

        else:
            if inNodes == 1:
                if len(line.split()) == 1:
                    nodes = np.zeros((int(line), 3))
                    if verbose:
                        print('  Nodes: %s' % int(line))
                else:
                    node_coordinates = np.array(line.split(), 'float')[1:]
                    if precision is None:
                        nodes[nCount, :] = node_coordinates
                    else:
                        nodes[nCount, :] = np.round(
                            node_coordinates, precision)
                    nCount += 1

            elif inElements == 1:
                if len(line.split()) == 1:
                    if verbose:
                        print('  Entries: %s' % int(line))
                    points, lines, triangles, tets = [], [], [], []
                    quads = []

                else:
                    # Element entries follow the following format:
                    # elm-number elm-type number-of-tags < tag > …
                    #   node-number-list

                    # strip elm-number here
                    entry = [int(e_) for e_ in line.split()][1:]

                    if entry[0] == 15:  # Points
                        # point node, marker (1st tag)
                        points.append((entry[-1], entry[2]))
                    elif entry[0] == 1:
                        lines.append((entry[-2], entry[-1], entry[2]))
                    elif entry[0] == 2:  # Tri
                        triangles.append((entry[-3], entry[-2], entry[-1],
                                          entry[2]))
                    elif entry[0] == 3:  # Quads
                        quads.append((entry[-4], entry[-3], entry[-2],
                                      entry[-1], entry[2]))
                    elif entry[0] == 4:  # Tet
                        tets.append((entry[-4], entry[-3], entry[-2],
                                     entry[-1], entry[2]))
                    elif entry[0] == 6:
                        pg.error(
                            "Quadrangles and prisms are not supported yet")


  Nodes: 66274


ValueError: could not convert string to float: '\x01\x00\x00\x00\x90¡*©lAÙ@¸F¬\x87¯:%Á\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00ø\x92láÜîà@,ÿÖÈz'

In [4]:
pg.show(mesh, ax=ax, markers=True, clipBoundaryMarkers=True)

Info    : Reading '/bettik/alberta/small-arctic-10km-sidfex/experiments/run3_330d_partnodisk/par32small_arctic_10km.msh'...
Info    : 69081 nodes
Info    : 136510 elements
Info    : Converting old partitioning...                                   
Info    : Creating partition topology...
Info    :  - Creating partition curves
Info    :  - Creating partition points
Info    : Done creating partition topology
Info    : Done reading '/bettik/alberta/small-arctic-10km-sidfex/experiments/run3_330d_partnodisk/par32small_arctic_10km.msh'
